In [1]:
import sqlite3
import spacy
import preprocessing
import numpy as np

# 0. Fetch data from db

In [2]:
con = sqlite3.connect("../../db/db_training.sqlite")

cur = con.cursor()

OperationalError: unable to open database file

In [ ]:
data_dict = {}
'''
Format: {[category]: list[page_text_content]}
'''
# sql = sql_utils.get_sql_statement("select_all_text.sql")
with open("../../db/sql/select_all_text.sql") as sql_file:
    sql = sql_file.read()
    sql_file.close()

cur.execute(sql)
response = cur.fetchall()

print(response)

# 1. Cleaning

# 2. Create a vocabulary

# 3. Feature extraction (BOW)

TODO:
- Try auto-sklearn
- Create model.py (with pytorch)